In [175]:
"""
denclue.py
@author: mgarrett
"""
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClusterMixin
import networkx as nx

def _hill_climb(x_t, X, W=None, h=0.1, eps=1e-7):
    """
    This function climbs the 'hill' of the kernel density function
    and finds the 'peak', which represents the density attractor
    """
    error = 99.
    prob = 0.
    x_l1 = np.copy(x_t)
    
    #Sum of the last three steps is used to establish radius
    #of neighborhood around attractor. Authors suggested two
    #steps works well, but I found three is more robust to
    #noisey datasets.
    radius_new = 0.
    radius_old = 0.
    radius_twiceold = 0.
    iters = 0.
    while True:
        radius_thriceold = radius_twiceold
        radius_twiceold = radius_old
        radius_old = radius_new
        x_l0 = np.copy(x_l1)
        x_l1, density = _step(x_l0, X, W=W, h=h)
        error = density - prob
        prob = density
        radius_new = np.linalg.norm(x_l1-x_l0)
        radius = radius_thriceold + radius_twiceold + radius_old + radius_new
        iters += 1
        if iters>3 and error < eps:
            break
    return [x_l1, prob, radius]

def _step(x_l0, X, W=None, h=0.1):
    n = X.shape[0]
    d = X.shape[1]
    superweight = 0. #superweight is the kernel X weight for each item
    x_l1 = np.zeros((1,d))
    if W is None:
        W = np.ones((n,1))
    else:
        W = W
    for j in range(n):
        kernel = kernelize(x_l0, X[j], h, d)
        kernel = kernel * W[j]/(h**d)
        superweight = superweight + kernel
        x_l1 = x_l1 + (kernel * X[j])
    x_l1 = x_l1/superweight
    density = superweight/np.sum(W)
    return [x_l1, density]
    
def kernelize(x, y, h, degree):
    kernel = np.exp(-(np.linalg.norm(x-y)/h)**2./2.)/((2.*np.pi)**(degree/2))
    return kernel

class DENCLUE(BaseEstimator, ClusterMixin):
    """Perform DENCLUE clustering from vector array.
    Parameters
    ----------
    h : float, optional
        The smoothing parameter for the gaussian kernel. This is a hyper-
        parameter, and the optimal value depends on data. Default is the
        np.std(X)/5.
    eps : float, optional
        Convergence threshold parameter for density attractors
    min_density : float, optional
        The minimum kernel density required for a cluster attractor to be
        considered a cluster and not noise.  Cluster info will stil be kept
        but the label for the corresponding instances will be -1 for noise.
        Since what consitutes a high enough kernel density depends on the
        nature of the data, it's often best to fit the model first and 
        explore the results before deciding on the min_density, which can be
        set later with the 'set_minimum_density' method.
        Default is 0.
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. In this version, I've only tested 'euclidean' at this
        moment.
    Attributes
    -------
    cluster_info_ : dictionary [n_clusters]
        Contains relevant information of all clusters (i.e. density attractors)
        Information is retained even if the attractor is lower than the
        minimum density required to be labelled a cluster.
    labels_ : array [n_samples]
        Cluster labels for each point.  Noisy samples are given the label -1.
    Notes
    -----
    References
    ----------
    Hinneburg A., Gabriel HH. "DENCLUE 2.0: Fast Clustering Based on Kernel 
    Density Estimation". In: R. Berthold M., Shawe-Taylor J., Lavrač N. (eds)
    Advances in Intelligent Data Analysis VII. IDA 2007
    """
    
    def __init__(self, h=None, eps=1e-8, min_density=0., metric='euclidean'):    
        print('h', h)
        self.h = h        
        self.eps = eps
        self.min_density = min_density
        self.metric = metric
        
    def fit(self, X, y=None, sample_weight=None):
        if not self.eps > 0.0:
            raise ValueError("eps must be positive.")
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        density_attractors = np.zeros((self.n_samples,self.n_features))
        radii = np.zeros((self.n_samples,1))
        density = np.zeros((self.n_samples,1))
        
        #create default values
        if self.h is None:
            self.h = np.std(X)/5
        if sample_weight is None:
            sample_weight = np.ones((self.n_samples,1))
        else:
            sample_weight = sample_weight
        
        #initialize all labels to noise
        labels = -np.ones(X.shape[0])
        
        #climb each hill
        for i in range(self.n_samples):
            density_attractors[i], density[i], radii[i] = _hill_climb(X[i], X, W=sample_weight,
                                                     h=self.h, eps=self.eps)
        
        #initialize cluster graph to finalize clusters. Networkx graph is
        #used to verify clusters, which are connected components of the
        #graph. Edges are defined as density attractors being in the same
        #neighborhood as defined by our radii for each attractor.
        cluster_info = {}
        num_clusters = 0
        cluster_info[num_clusters]={'instances': [0],
                                    'centroid': np.atleast_2d(density_attractors[0])}
        
        g_clusters = nx.Graph()
        for j1 in range(self.n_samples):
            g_clusters.add_node(j1)
            g_clusters.node[j1]['attractor'] = density_attractors[j1]
            g_clusters.node[j1]['radius'] = radii[j1]
            g_clusters.node[j1]['density'] = density[j1]
#=========================================================================================================        
        #populate cluster graph
        for j1 in range(self.n_samples):
            for j2 in (x for x in range(self.n_samples) if x != j1):
                if g_clusters.has_edge(j1,j2):
                    continue
                diff = np.linalg.norm(g_clusters.node[j1]['attractor']-
                                      g_clusters.node[j2]['attractor'])
                if diff <= (g_clusters.node[j1]['radius']+g_clusters.node[j1]['radius']):
                    g_clusters.add_edge(j1, j2)
                    
        #connected components represent a cluster
        clusters = list(nx.connected_component_subgraphs(g_clusters))
        num_clusters = 0
        
        #loop through all connected components
        for clust in clusters:
            
            #get maximum density of attractors and location
            max_instance = max(clust, key=lambda x: clust.node[x]['density'])
            max_density = clust.node[max_instance]['density']
            max_centroid = clust.node[max_instance]['attractor']
            
            #populate cluster_info dict
            cluster_info[num_clusters] = {'instances': clust.nodes(),
                                        'size': len(clust.nodes()),
                                        'centroid': max_centroid,
                                        'density': max_density}
            
            #if the cluster density is not higher than the minimum,
            #instances are kept classified as noise
            if max_density >= self.min_density:
                labels[clust.nodes()]=num_clusters            
            num_clusters += 1
            
        self.clust_info_ = cluster_info
        self.labels_ = labels
        return self.clust_info_

In [181]:
data = pd.read_csv('iris.txt', header=None)
print(data.head(10))
data1 = np.array(data)
iris = np.mat(data1[:,0:4])

     0    1    2    3                4
0  5.9  3.0  4.2  1.5  Iris-versicolor
1  6.9  3.1  4.9  1.5  Iris-versicolor
2  6.6  2.9  4.6  1.3  Iris-versicolor
3  4.6  3.2  1.4  0.2      Iris-setosa
4  6.0  2.2  4.0  1.0  Iris-versicolor
5  4.7  3.2  1.3  0.2      Iris-setosa
6  6.5  3.0  5.8  2.2   Iris-virginica
7  5.8  2.7  5.1  1.9   Iris-virginica
8  6.7  3.1  5.6  2.4   Iris-virginica
9  6.7  2.5  5.8  1.8   Iris-virginica


In [177]:
den = DENCLUE(h=0.37, eps=0.0001, min_density=0.13, metric='euclidean')
clusters = den.fit(iris, y=None, sample_weight=None)
print(clusters)

clu = pd.DataFrame.from_dict(clusters, orient='index')
print(clu)

    

h 0.37
{0: {'instances': NodeView((0, 4, 139, 12, 17, 146, 23, 24, 26, 33, 34, 35, 36, 44, 47, 50, 59, 64, 65, 72, 81, 85, 96, 98, 110, 114, 126)), 'size': 27, 'centroid': array([5.91118336, 2.84759438, 4.37154782, 1.37436614]), 'density': array([0.13773153])}, 1: {'instances': NodeView((128, 1, 129, 2, 131, 132, 6, 7, 8, 9, 137, 138, 140, 13, 141, 142, 143, 144, 19, 20, 21, 147, 148, 28, 29, 32, 38, 39, 40, 41, 42, 43, 46, 51, 53, 55, 56, 57, 58, 60, 61, 66, 70, 71, 77, 79, 80, 82, 83, 84, 86, 87, 89, 92, 93, 94, 95, 97, 99, 100, 101, 102, 103, 104, 107, 112, 116, 117, 118, 119, 120, 122, 123)), 'size': 73, 'centroid': array([6.18638554, 2.89567907, 4.72334636, 1.56653329]), 'density': array([0.14536436])}, 2: {'instances': NodeView((130, 3, 5, 134, 133, 135, 136, 10, 11, 14, 15, 16, 145, 18, 149, 22, 25, 27, 30, 31, 37, 45, 48, 49, 52, 54, 62, 63, 67, 68, 69, 73, 74, 75, 76, 78, 88, 90, 91, 105, 106, 108, 109, 111, 113, 115, 121, 124, 125, 127)), 'size': 50, 'centroid': array([4.9757

In [191]:
major_class = 0
t_size = 0
for clust in clusters:
    n_ver = 0
    n_set = 0
    n_vir = 0
    for index, row in data.iterrows():
        if index in clusters[clust]['instances']:
            if row[4] == 'Iris-versicolor':
                n_ver += 1
            elif row[4] == 'Iris-setosa':
                n_set += 1
            elif row[4] == 'Iris-virginica':
                n_vir += 1

    major_class += max(n_ver, n_set, n_vir)
    t_size += clusters[clust]['size']

print('Purity = ', round(major_class/t_size, 2))

Purity =  0.83
